In [ ]:
pip install alibi[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 445 kB 4.4 MB/s 
     |████████████████████████████████| 4.2 MB 79.3 MB/s 
     |████████████████████████████████| 28.0 MB 1.2 MB/s 
     |████████████████████████████████| 6.6 MB 42.9 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 84.6 MB/s 
  Created wheel for spacy-lookups-data: filename=spacy_lookups_data-0.1.0-py2.py3-none-any.whl size=28052158 sha256=d834403ce5fbd634b0c7632c109ee186705b9b94e0a3d608f4dff9b84782b883
  Stored in directory: /root/.cache/pip/wheels/b6/09/83/36dd0224ce32dcdf5e218b36362235ca2e50cece60a966ae1b
Successfully built spacy-lookups-data
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import os
import numpy as np
import matplotlib as mpl
import matplotlib.cm
import tensorflow as tf
import tensorflow.keras as keras

from tqdm import tqdm
from typing import Optional, Union, List, Dict
from IPython.display import HTML
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from transformers import PreTrainedTokenizer
from alibi.explainers import IntegratedGradients

In [ ]:
def decode_sentence(x: List[int], reverse_index: Dict[int, str], unk_token: str = '[UNK]') -> str:
    """ 
    Decodes the tokenized sentences from keras IMDB dataset into plain text.
    
    Parameters
    ----------
    x
        List of integers to be docoded.
    revese_index:
        Reverse index map, from `int` to `str`.
    unk_token:
        Unkown token to be used.
        
    Returns
    -------
        Decoded sentence.
    """
    # the `-3` offset is due to the special tokens used by keras
    # see https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
    return " ".join([reverse_index.get(i - 3, unk_token) for i in x])


def process_sentences(sentence: List[str], 
                      tokenizer: PreTrainedTokenizer, 
                      max_len: int) -> Dict[str, np.ndarray]:
    """
    Tokenize the text sentences.
    
    Parameters
    ----------
    sentence:
        Sentence to be processed.
    tokenizer:
        Tokenizer to be used.
    
    Returns
    -------
        Tokenized representation containing:
         - input_ids
         - attention_mask
    """
    # since we are using the model for classification, we need to include special char (i.e, '[CLS]', ''[SEP]')
    # check the example here: https://huggingface.co/transformers/v4.4.2/quicktour.html
    z = tokenizer(sentence, 
                  add_special_tokens=True, 
                  padding='max_length', 
                  max_length=max_len, 
                  truncation=True,
                  return_attention_mask = True,  
                  return_tensors='np')
    return z

In [ ]:
def  hlstr(string: str , color: str = 'white') -> str:
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"


def colorize(attrs: np.ndarray, cmap: str = 'PiYG') -> List:
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    
    Parameters
    ----------
    attrs:
        Attributions to be visualized.
    cmap:
        Matplotlib cmap type.
    """
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)
    return list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))


def display(X: np.ndarray, 
            attrs: np.ndarray, 
            tokenizer: PreTrainedTokenizer,
            pred: np.ndarray) -> None:
    """
    Display the attribution of a given instance.
    
    Parameters
    ----------
    X:
        Instance to display the attributions for.
    attrs:
        Attributions values for the given instance.
    tokenizer:
        Tokenizer to be used for decoding.
    pred:
        Classification label (prediction) for the given instance.
    """
    pred_dict = {1: 'Positive review', 0: 'Negative review'}
    
    # remove padding
    fst_pad_indices = np.where(X ==tokenizer.pad_token_id)[0]
    if len(fst_pad_indices) > 0:
        X, attrs = X[:fst_pad_indices[0]], attrs[:fst_pad_indices[0]]
    
    # decode tokens and get colors
    tokens = [tokenizer.decode([X[i]]) for i in range(len(X))]
    colors = colorize(attrs)
    
    print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))
    return HTML("".join(list(map(hlstr, tokens, colors))))

In [ ]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = TFAutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# # load model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [ ]:
class AutoModelWrapper(keras.Model):
    def __init__(self, transformer: keras.Model, **kwargs):
        """
        Constructor.
        
        Parameters
        ----------
        transformer:
            Transformer to be wrapped.
        """
        super().__init__()
        self.transformer = transformer

    def call(self, 
             input_ids: Union[np.ndarray, tf.Tensor], 
             attention_mask: Optional[Union[np.ndarray, tf.Tensor]] = None,
             training: bool = False):
        """
        Performs forward pass throguh the model.
        
        Parameters
        ----------
        input_ids:
            Indices of input sequence tokens in the vocabulary.
        attention_mask:
            Mask to avoid performing attention on padding token indices.
        
        Returns
        -------
            Classification probabilities.
        """
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask, training=training)
        return tf.nn.softmax(out.logits, axis=-1)
    
    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
auto_model = AutoModelWrapper(model)


In [ ]:
print(auto_model)

In [ ]:
max_features = 10000
max_len = 128

In [ ]:
text_samples = ['I love you, I like you', 
                'I love you, I like you, but I also kind of dislike you',
                'Everything is so nice about you']

# since using the uncased model, we need to use lowercase sentences
text_samples = [text.lower() for text in text_samples]

# tokenize the sentences using the transformer's tokenizer.
tokenized_samples = process_sentences(text_samples, tokenizer, max_len)
X_test = tokenized_samples['input_ids'].astype(np.int32)

# the values of the kwargs have to be `tf.Tensor`. 
# see transformers issue #14404: https://github.com/huggingface/transformers/issues/14404
kwargs = {k: tf.constant(v) for k,v in tokenized_samples.items() if k == 'attention_mask'}

In [ ]:
auto_model.layers[0].layers

In [ ]:
layer = auto_model.layers[0].layers[0].embeddings

In [ ]:
n_steps = 50
internal_batch_size = 5
method = "gausslegendre"

ig  = IntegratedGradients(auto_model,
                          layer=layer,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
predictions = auto_model(X_test, **kwargs).numpy().argmax(axis=1)

# Get the baselines. Note that the baseline contains special characters (e.g, [CLS], [SEP], [UNK] [PAD]) and
# the regular tokens are replaced by the [PAD] token which is a neutral token.
# By including special tokens such as [CLS], [SEP], [UNK], we ensure that the attribution for those tokens
# will be 0 if we use the embedding layer. The 0 attribution is due to integration between [x, x] which is 0.
mask = np.isin(X_test, tokenizer.all_special_ids)
baselines = X_test * mask + tokenizer.pad_token_id * (1 - mask)

# get explanation
explanation = ig.explain(X_test, 
                         forward_kwargs=kwargs,
                         baselines=baselines, 
                         target=predictions)

In [ ]:
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (3, 128, 768)


In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (3, 128)


In [ ]:
index = 0
display(X=X_test[index], attrs=attrs[index], pred=predictions[index], tokenizer=tokenizer)

Predicted label =  0: Negative review


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("FINBERT-allData (1).csv", encoding="latin", header=[0])
text=df['Description']
label=df['Semantic']

In [ ]:
s=text.head(20)
print(s[3])

With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .


In [ ]:
s = [text.lower() for text in s]

# tokenize the sentences using the transformer's tokenizer.
tokenized_samples = process_sentences(s, tokenizer, max_len)
X_test = tokenized_samples['input_ids'].astype(np.int32)

# the values of the kwargs have to be `tf.Tensor`. 
# see transformers issue #14404: https://github.com/huggingface/transformers/issues/14404
kwargs = {k: tf.constant(v) for k,v in tokenized_samples.items() if k == 'attention_mask'}

In [ ]:
print(s)

['according to gran , the company has no plans to move all production to russia , although that is where the company is growing .', 'technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .', 'the international electronic industry company elcoteq has laid off tens of employees from its tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily postimees reported .', 'with the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .', "according to the company 's updated strategy for the years 2009-2012 , basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .", "financing of aspocomp 's

In [ ]:
auto_model.layers[0].layers

In [ ]:
layer = auto_model.layers[0].layers[0].embeddings

In [ ]:
n_steps = 50
internal_batch_size = 5
method = "gausslegendre"

ig  = IntegratedGradients(auto_model,
                          layer=layer,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
predictions = auto_model(X_test, **kwargs).numpy().argmax(axis=1)

# Get the baselines. Note that the baseline contains special characters (e.g, [CLS], [SEP], [UNK] [PAD]) and
# the regular tokens are replaced by the [PAD] token which is a neutral token.
# By including special tokens such as [CLS], [SEP], [UNK], we ensure that the attribution for those tokens
# will be 0 if we use the embedding layer. The 0 attribution is due to integration between [x, x] which is 0.
mask = np.isin(X_test, tokenizer.all_special_ids)
baselines = X_test * mask + tokenizer.pad_token_id * (1 - mask)

# get explanation
explanation = ig.explain(X_test, 
                         forward_kwargs=kwargs,
                         baselines=baselines, 
                         target=predictions)

In [ ]:
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128, 768)


In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128)


In [ ]:
index = 3
display(X=X_test[index], attrs=attrs[index], pred=predictions[index], tokenizer=tokenizer)

Predicted label =  1: Positive review


In [ ]:
# print(s[3])
s[3]=s[3].replace("increase", "adjust")
print(s[3])


with the new production plant the company would adjust its capacity to meet the expected adjust in demand and would improve the use of raw materials and therefore adjust the production profitability .


In [ ]:
text[3]=s[3]
print(text[3])

with the new production plant the company would adjust its capacity to meet the expected adjust in demand and would improve the use of raw materials and therefore adjust the production profitability .


In [ ]:
s = [text.lower() for text in s]

# tokenize the sentences using the transformer's tokenizer.
tokenized_samples = process_sentences(s, tokenizer, max_len)
X_test = tokenized_samples['input_ids'].astype(np.int32)

# the values of the kwargs have to be `tf.Tensor`. 
# see transformers issue #14404: https://github.com/huggingface/transformers/issues/14404
kwargs = {k: tf.constant(v) for k,v in tokenized_samples.items() if k == 'attention_mask'}

In [ ]:
auto_model.layers[0].layers

In [ ]:
layer = auto_model.layers[0].layers[0].embeddings

In [ ]:
n_steps = 50
internal_batch_size = 5
method = "gausslegendre"

ig  = IntegratedGradients(auto_model,
                          layer=layer,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
predictions = auto_model(X_test, **kwargs).numpy().argmax(axis=1)

# Get the baselines. Note that the baseline contains special characters (e.g, [CLS], [SEP], [UNK] [PAD]) and
# the regular tokens are replaced by the [PAD] token which is a neutral token.
# By including special tokens such as [CLS], [SEP], [UNK], we ensure that the attribution for those tokens
# will be 0 if we use the embedding layer. The 0 attribution is due to integration between [x, x] which is 0.
mask = np.isin(X_test, tokenizer.all_special_ids)
baselines = X_test * mask + tokenizer.pad_token_id * (1 - mask)

# get explanation
explanation = ig.explain(X_test, 
                         forward_kwargs=kwargs,
                         baselines=baselines, 
                         target=predictions)

In [ ]:
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128, 768)


In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128)


In [ ]:
index = 3
display(X=X_test[index], attrs=attrs[index], pred=predictions[index], tokenizer=tokenizer)

Predicted label =  0: Negative review


In [ ]:
# print(s[3])
s[3]=s[3].replace("adjust", "decrease")
print(s[3])


with the new production plant the company would decrease its capacity to meet the expected decrease in demand and would improve the use of raw materials and therefore decrease the production profitability .


In [ ]:
text[3]=s[3]
print(text[3])

with the new production plant the company would decrease its capacity to meet the expected decrease in demand and would improve the use of raw materials and therefore decrease the production profitability .


In [ ]:
s = [text.lower() for text in s]

# tokenize the sentences using the transformer's tokenizer.
tokenized_samples = process_sentences(s, tokenizer, max_len)
X_test = tokenized_samples['input_ids'].astype(np.int32)

# the values of the kwargs have to be `tf.Tensor`. 
# see transformers issue #14404: https://github.com/huggingface/transformers/issues/14404
kwargs = {k: tf.constant(v) for k,v in tokenized_samples.items() if k == 'attention_mask'}

In [ ]:
auto_model.layers[0].layers

In [ ]:
layer = auto_model.layers[0].layers[0].embeddings

In [ ]:
n_steps = 50
internal_batch_size = 5
method = "gausslegendre"

ig  = IntegratedGradients(auto_model,
                          layer=layer,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
predictions = auto_model(X_test, **kwargs).numpy().argmax(axis=1)

# Get the baselines. Note that the baseline contains special characters (e.g, [CLS], [SEP], [UNK] [PAD]) and
# the regular tokens are replaced by the [PAD] token which is a neutral token.
# By including special tokens such as [CLS], [SEP], [UNK], we ensure that the attribution for those tokens
# will be 0 if we use the embedding layer. The 0 attribution is due to integration between [x, x] which is 0.
mask = np.isin(X_test, tokenizer.all_special_ids)
baselines = X_test * mask + tokenizer.pad_token_id * (1 - mask)

# get explanation
explanation = ig.explain(X_test, 
                         forward_kwargs=kwargs,
                         baselines=baselines, 
                         target=predictions)

In [ ]:
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128, 768)


In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (20, 128)


In [ ]:
index = 3
display(X=X_test[index], attrs=attrs[index], pred=predictions[index], tokenizer=tokenizer)

KeyError: ignored